# Optimization of Heterogeneous Mini-Batches

In [7]:
import numpy
import open3d

mesh_files = ["cube.obj", "octahedron1.obj", "octahedron2.obj", "dodecahedron.obj"]

## Load mesh and visualize it using Open3D
print("Visualizing mesh using Open3D")
# Set up the camera intrinsic for orthographic approximation <-- doesn't work
# width, height = 800, 800  # Resolution of the viewport
# intrinsic = open3d.camera.PinholeCameraIntrinsic(width, height, fx=1000, fy=1000, cx=width / 2, cy=height / 2)

# # Create extrinsic (camera position and orientation)
# extrinsic = numpy.eye(4)
# extrinsic[2, 3] = -5  # Move the camera back a bit to view the object

# # Combine the intrinsic and extrinsic into camera parameters
# camera_params = open3d.camera.PinholeCameraParameters()
# camera_params.intrinsic = intrinsic
# camera_params.extrinsic = extrinsic

# # Custom draw function to apply camera settings and render options
# def custom_draw(vis):
#     # Set camera parameters
#     ctr = vis.get_view_control()
#     ctr.convert_from_pinhole_camera_parameters(camera_params)

#     # Set render options for wireframe and back face
#     render_option = vis.get_render_option()
#     render_option.mesh_show_wireframe = True
#     render_option.mesh_show_back_face = True

#     return False

for mesh_file in mesh_files:
    mesh = open3d.io.read_triangle_mesh(mesh_file, enable_post_processing=True)
    open3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True, mesh_show_back_face=True)
    # open3d.visualization.draw_geometries_with_animation_callback([mesh], custom_draw)

Visualizing mesh using Open3D


In [ ]:
import numpy
import torch

from pytorch3d.io import load_objs_as_meshes
from pytorch3d.structures.meshes import join_meshes_as_batch
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import chamfer_distance

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

mesh_files = ["cube.obj", "octahedron1.obj", "octahedron2.obj", "dodecahedron.obj"]

mesh_list = list()
for mesh_file in mesh_files:
    mesh = load_objs_as_meshes([mesh_file], device=device)
    mesh_list.append(mesh)

mesh_batch = join_meshes_as_batch(mesh_list, include_textures=False)
print(f"Mini-Batch of meshes: {mesh_batch}")

vertex_list = mesh_batch.verts_list()
face_list = mesh_batch.faces_list()

print(f"Vertex list: {vertex_list}")
print(f"Face list: {face_list}")

vertex_padded = mesh_batch.verts_padded()
face_padded = mesh_batch.faces_padded()

print(f"Vertex padded: {vertex_padded}")
print(f"Face padded: {face_padded}")

vertex_packed = mesh_batch.verts_packed()
num_vertices = vertex_packed.shape[0]
face_packed = mesh_batch.faces_packed()

print(f"Vertex packed: {vertex_packed}")
print(f"Numbef of vertices: {num_vertices}")
print(f"Face packed: {face_packed}")

mesh_batch_noisy = mesh_batch.clone()

motion_gt = numpy.array([3, 4, 5])
motion_gt = torch.as_tensor(motion_gt)
print(f"Motion Ground Truth: {motion_gt}")

motion_gt = motion_gt[None, :]
motion_gt = motion_gt.to(device)
print(f"Motion Ground Truth: {motion_gt}")

noise = (0.1**0.5) * torch.randn(mesh_batch_noisy.verts_packed().shape).to(device)
# motion_gt = numpy.array([3, 4, 5])
# motion_gt = torch.as_tensor(motion_gt)
# motion_gt = motion_gt.to(device)
noise = noise + motion_gt
mesh_batch_noisy = mesh_batch_noisy.offset_verts(noise).detach()

motion_estimate = torch.zeros(motion_gt.shape, device=device, requires_grad=True)
# optimizer = torch.optim.Adam([motion_estimate], lr=0.1)
# optimizer = torch.optim.AdamW([motion_estimate], lr=0.1)
optimizer = torch.optim.SGD([motion_estimate], lr=0.1, momentum=0.9)

for i in range(0, 2000):
    optimizer.zero_grad()
    current_mesh_batch = mesh_batch.offset_verts(motion_estimate.repeat(num_vertices, 1))

    sample_trg = sample_points_from_meshes(current_mesh_batch, 5000)
    sample_src = sample_points_from_meshes(mesh_batch_noisy, 5000)
    loss, _ = chamfer_distance(sample_trg, sample_src)

    loss.backward()
    optimizer.step()
    print(f"I: {i}; Motion Estimation: {motion_estimate}")
# print(f"Final Motion Estimation: {motion_estimate}")

Mini-Batch of meshes: <pytorch3d.structures.meshes.Meshes object at 0x00000261716241D0>
Vertex list: [tensor([[-0.5000, -0.5000,  0.5000],
        [-0.5000, -0.5000, -0.5000],
        [-0.5000,  0.5000, -0.5000],
        [-0.5000,  0.5000,  0.5000],
        [ 0.5000, -0.5000,  0.5000],
        [ 0.5000, -0.5000, -0.5000],
        [ 0.5000,  0.5000, -0.5000],
        [ 0.5000,  0.5000,  0.5000]], device='cuda:0'), tensor([[  0.,   0.,  90.],
        [ 45.,  45.,   0.],
        [ 45., -45.,   0.],
        [-45., -45.,   0.],
        [-45.,  45.,   0.],
        [  0.,   0., -90.]], device='cuda:0'), tensor([[  0.,   0.,  10.],
        [  0.,  10.,   0.],
        [-10.,   0.,   0.],
        [  0., -10.,   0.],
        [ 10.,   0.,   0.],
        [  0.,   0., -10.]], device='cuda:0'), tensor([[-0.5774, -0.5774,  0.5774],
        [ 0.9342,  0.3568,  0.0000],
        [ 0.9342, -0.3568,  0.0000],
        [-0.9342,  0.3568,  0.0000],
        [-0.9342, -0.3568,  0.0000],
        [ 0.0000,  0.934